In [1]:
import pyspark
from pyspark.sql import SparkSession

spark= SparkSession.builder \
    .master("local") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/22 08:16:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green= spark.read.parquet('data/pq/green/*/*')

In [3]:
pyspark.__version__

'3.4.1'

In [4]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [5]:
df_green.createOrReplaceTempView('green')

In [6]:
from pyspark.sql.functions import date_trunc

In [7]:
df_green_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS zone,
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
   
    -- Revenue calculation 
    SUM(tolls_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")
# Doing 1,2,3 provides the selection of top 3 params.

In [8]:
df_green_result.show()

+----+-------------------+------+--------------+
|zone|               hour|amount|number_records|
+----+-------------------+------+--------------+
|   1|2020-01-01 03:00:00|   0.0|             1|
|   1|2020-01-03 05:00:00|   0.0|             1|
|   1|2020-01-26 10:00:00|   0.0|             1|
|   1|2020-01-29 18:00:00|   0.0|             1|
|   1|2020-02-01 06:00:00|   0.0|             1|
|   1|2020-02-06 07:00:00|   0.0|             1|
|   1|2020-02-16 07:00:00|   0.0|             1|
|   1|2020-02-17 13:00:00|   0.0|             1|
|   1|2020-02-26 17:00:00|   0.0|             1|
|   1|2020-03-02 15:00:00|   0.0|             1|
|   1|2020-03-11 12:00:00|   0.0|             1|
|   1|2020-03-12 11:00:00|   0.0|             1|
|   1|2020-03-14 11:00:00|   0.0|             1|
|   1|2020-03-21 17:00:00|   0.0|             1|
|   1|2020-04-10 13:00:00|   0.0|             1|
|   1|2020-04-23 08:00:00|   0.0|             1|
|   1|2020-07-12 07:00:00|  10.5|             1|
|   1|2020-07-15 06:

In [9]:
df_green_result.write.parquet('data/report/revenue/green', mode='overwrite')

In [10]:
df_yellow= spark.read.parquet('data/pq/yellow/*/*')
df_yellow.createOrReplaceTempView('yellow')

In [11]:
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [12]:
df_yellow_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS zone,
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
   
    -- Revenue calculation 
    SUM(tolls_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")
# Doing 1,2,3 provides the selection of top 3 params.

In [13]:
df_yellow_result.show()

+----+-------------------+------+--------------+
|zone|               hour|amount|number_records|
+----+-------------------+------+--------------+
|   1|2020-01-01 03:00:00|   0.0|             1|
|   1|2020-01-01 04:00:00|   0.0|             1|
|   1|2020-01-01 05:00:00|   0.0|             1|
|   1|2020-01-01 06:00:00|  43.5|             4|
|   1|2020-01-01 07:00:00|   0.0|             1|
|   1|2020-01-01 08:00:00|   0.0|             1|
|   1|2020-01-01 09:00:00|   0.0|             2|
|   1|2020-01-01 10:00:00|   0.0|             2|
|   1|2020-01-01 11:00:00|   0.0|             1|
|   1|2020-01-01 12:00:00|   0.0|             3|
|   1|2020-01-01 13:00:00|  25.0|             2|
|   1|2020-01-01 14:00:00|  17.0|             3|
|   1|2020-01-01 15:00:00|  15.5|             4|
|   1|2020-01-01 16:00:00|  21.0|             6|
|   1|2020-01-01 17:00:00|  17.5|             3|
|   1|2020-01-01 18:00:00|   0.0|             3|
|   1|2020-01-01 19:00:00|   0.0|             1|
|   1|2020-01-01 20:

In [26]:
df_yellow_result \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [27]:
#Instead of working all on the files we will look onto materialized results.
df_green_result = spark.read.parquet('data/report/revenue/green')
df_yellow_result = spark.read.parquet('data/report/revenue/yellow')

In [28]:
#Creating So That Both Will have distinct records and amount.
df_green_result_tmp=df_green_result\
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records','green_number_records')

df_yellow_result_tmp = df_yellow_result \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records','yellow_number_records')

In [29]:
#Doing Outer Join.
df_join= df_green_result_tmp.join(df_yellow_result_tmp, on=['hour','zone'], how='outer')

In [32]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')
#While seeing results of computation there was not much difference using spark read or otherwise.
#It may be because we are doing join and group by in the same file.
#This maybe because the dataset being small otherwise this is a good approach.

In [33]:
df_join.show()

+-------------------+----+------------+--------------------+-------------+---------------------+
|               hour|zone|green_amount|green_number_records|yellow_amount|yellow_number_records|
+-------------------+----+------------+--------------------+-------------+---------------------+
|2020-01-01 00:00:00|   3|        null|                null|          0.0|                    1|
|2020-01-01 00:00:00|   4|        null|                null|         6.12|                   57|
|2020-01-01 00:00:00|   7|        6.12|                  45|        12.24|                   38|
|2020-01-01 00:00:00|  10|        null|                null|          0.0|                    2|
|2020-01-01 00:00:00|  12|        null|                null|          0.0|                    6|
|2020-01-01 00:00:00|  13|        null|                null|         6.12|                   56|
|2020-01-01 00:00:00|  14|        null|                null|          0.0|                    1|
|2020-01-01 00:00:00|  15|    

In [34]:
df_join = spark.read.parquet('data/report/revenue/total')

In [36]:
df_zones = spark.read.parquet('zones/')

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/home/abinash/code/zones.

In [ ]:
df_zones.show()

In [35]:
df_join.show()

+-------------------+----+------------+--------------------+-------------+---------------------+
|               hour|zone|green_amount|green_number_records|yellow_amount|yellow_number_records|
+-------------------+----+------------+--------------------+-------------+---------------------+
|2020-01-01 00:00:00|   3|        null|                null|          0.0|                    1|
|2020-01-01 00:00:00|   4|        null|                null|         6.12|                   57|
|2020-01-01 00:00:00|   7|        6.12|                  45|        12.24|                   38|
|2020-01-01 00:00:00|  10|        null|                null|          0.0|                    2|
|2020-01-01 00:00:00|  12|        null|                null|          0.0|                    6|
|2020-01-01 00:00:00|  13|        null|                null|         6.12|                   56|
|2020-01-01 00:00:00|  14|        null|                null|          0.0|                    1|
|2020-01-01 00:00:00|  15|    